In [1]:

%load_ext sparkmagic.magics

#Livy endpoint http://10.0.123.114:8998


In [2]:

!klist

Ticket cache: FILE:/tmp/krb5cc_12574
Default principal: dominospark@KDCDOMINO.COM

Valid starting       Expires              Service principal
08/18/2021 12:37:58  08/18/2021 22:37:58  krbtgt/KDCDOMINO.COM@KDCDOMINO.COM
	renew until 08/19/2021 12:37:58
08/18/2021 13:13:20  08/18/2021 22:37:58  HTTP/ip-10-0-123-114.us-west-2.compute.internal@KDCDOMINO.COM
	renew until 08/19/2021 12:37:58


In [3]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

In [4]:
%spark?

In [7]:
%spark 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
%%spark
numbers = sc.parallelize([1, 2, 3, 4])
print('First element of numbers is {} and its description is:\n{}'.format(numbers.first(), numbers.toDebugString()))
print('Count of numbers is {} \n{}'.format(numbers.count(), numbers.toDebugString()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://10.0.123.114:8998/sessions/18/statements/2 with error payload: {"msg":"requirement failed: Session isn't active."}


In [9]:
%spark info

Info for running Spark:
    Sessions:
        Name: session-name	Session id: 18	YARN id: application_1628875993215_0046	Kind: pyspark	State: idle
	Spark UI: http://ip-10-0-123-114.us-west-2.compute.internal:20888/proxy/application_1628875993215_0046/
	Driver Log: http://ip-10-0-113-154.us-west-2.compute.internal:8042/node/containerlogs/container_1628875993215_0046_01_000001/dominospark
    Session configs:
        {'driverMemory': '1000M', 'executorCores': 2}



In [7]:
%%spark -s spark-session
##Only if you have multiple spark sessions or else simple %%spark works
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
%%spark -s spark-session
##Only if you have multiple spark sessions or else simple %%spark works

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

columns = StructType([ StructField("name", StringType(), True),
                      StructField("value", DoubleType(), True)
                    ])

count = sc.parallelize(range(0, 1000),1) \
             .filter(inside).count()
data = [("Pi",4.0 * count/1000)]

df = spark.createDataFrame(data=data, schema=columns)

df.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|name|value|
+----+-----+
|  Pi|3.204|
+----+-----+

In [9]:
!/mnt/code/scripts/my_hdfs.sh dfs -rmr /user/dominospark/large-data-livy*

rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted /user/dominospark/large-data-livy
Deleted /user/dominospark/large-data-livy-2
Deleted /user/dominospark/large-data-livy-3
Deleted /user/dominospark/large-data-livy-4


In [10]:
%%spark

hdfs_path='/user/dominospark/large-data-livy'
data = []
for i in range(1000):
    data.append((i,random.randint(1,100),random.randint(1,100),random.randint(1,100)))

columns = StructType([ StructField("id", IntegerType(), True),
                       StructField("v1", IntegerType(), True),
                       StructField("v2", IntegerType(), True),
                       StructField("v3", IntegerType(), True) ])

df = spark.createDataFrame(data=data, schema=columns)
df.write.csv(hdfs_path)

df_load = spark.read.csv( hdfs_path, schema=columns)
df_load.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|500| 98| 79| 16|
|501| 66| 28| 27|
|502| 26| 92|  3|
|503| 90|  7| 28|
|504| 56| 31| 97|
|505| 42| 34| 48|
|506|  7| 40| 54|
|507| 19| 55| 24|
|508| 94|  6|  4|
|509|  9| 28| 33|
|510| 28| 57| 83|
|511| 25|  7| 32|
|512| 36| 51| 26|
|513| 73|  6| 83|
|514| 94|  3| 59|
|515|100| 15| 62|
|516| 31| 20| 35|
|517|  5| 86| 49|
|518| 59| 79| 53|
|519| 47| 22| 57|
+---+---+---+---+
only showing top 20 rows